# models

> Model and architecture tooling

In [ ]:
#|default_exp models

In [ ]:
#|hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from isaacai.utils import *
from isaacai.dataloaders import *

import matplotlib.pyplot as plt,matplotlib as mpl
import torchvision.transforms.functional as TF,torch.nn.functional as F
import fastcore.all as fc
import torch
from torch import nn, Tensor
from datasets import load_dataset, Dataset
from torch.utils.data import DataLoader
import pandas as pd, numpy as np

In [ ]:
torch.set_printoptions(precision=2, linewidth=140, sci_mode=False)
torch.manual_seed(1)
mpl.rcParams['image.cmap'] = 'gray'

import logging
logging.disable(logging.WARNING)

set_seed(42)

In [ ]:
xmean,xstd = 0.28, 0.35

@inplace
def transformi(b): b['image'] = [(TF.to_tensor(o)-xmean)/xstd for o in b['image']]

_dataset = load_dataset('fashion_mnist').with_transform(transformi)
_dataset = sample_dataset_dict(_dataset)

dls = DataLoaders.from_dataset_dict(_dataset, 64, num_workers=4)

  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
#| export models
class SimpleNet(nn.Module):
    ## simplified from Pytorch Tutorial
    def __init__(self,n_inp,n_hidden,n_out):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(n_inp, n_hidden),
            nn.ReLU(),
            nn.Linear(n_hidden, n_out)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [ ]:
model = SimpleNet(28*28,64,10)
fc.test_eq(model(fc.first(dls.train)[0]).shape,(64,10))

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()